In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 

# Step 1: Create a session and load the page
'''
url1 = 'https://www.nba.com/stats/players/pullup?PerMode=Totals'
url2 = 'https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
url3 = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 = 'https://www.nba.com/stats/players/transition?PerMode=Totals&dir=D&sort=POSS'
'''
cs ='https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
pullup ='https://www.nba.com/stats/players/pullup?PerMode=Totals'

touches = 'https://www.nba.com/stats/players/touches?PerMode=Totals'
drives = 'https://www.nba.com/stats/players/drives?PerMode=Totals'

wide_open = 'https://www.nba.com/stats/players/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
close = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals&dir=D&sort=PLUSMINUS'


In [2]:
#url_list = [cs,pullup]
from selenium.webdriver.support.select import Select

def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True


def get_ptables(url_list,path_list):
    data = []
    for i in range(len(url_list)):
        url = url_list[i]
        xpath = path_list[i]
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(8)
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        
        dropdown1 = Select(driver.find_element(By.XPATH, xpath))
        dropdown1.select_by_index(0)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')
        

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))
        #print(dfs)

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        driver.close()
        #return dfs
        df= dfs[-1]
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
       
        data.append(df)
    return data

In [3]:


url_list = [drives,wide_open,close,touches,cs,pullup]
name_list = ['drives','wide_open','close_6','touches','cs','pullup']
xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'
#xpath2 = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'
path_list = [xpath for i in range(len(url_list))]

In [4]:
tables= get_ptables(url_list,path_list)
tables

https://www.nba.com/stats/players/drives?PerMode=Totals
https://www.nba.com/stats/players/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals
https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals&dir=D&sort=PLUSMINUS
https://www.nba.com/stats/players/touches?PerMode=Totals
https://www.nba.com/stats/players/catch-shoot?PerMode=Totals
https://www.nba.com/stats/players/pullup?PerMode=Totals


[              PLAYER TEAM  GP   W   L  MIN  DRIVES  FGM  FGA   FG%  ...  PTS  \
 0        A.J. Lawson  MIN   1   1   0    2       0    0    0   0.0  ...    0   
 1           AJ Green  MIL   7   7   0   37       1    0    0   0.0  ...    0   
 2         AJ Griffin  ATL  19   9  10  356      45   15   27  55.6  ...   35   
 3       Aaron Gordon  DEN  20  13   7  584      55   14   23  60.9  ...   42   
 4      Aaron Holiday  ATL  21  13   8  359      71   12   34  35.3  ...   29   
 ..               ...  ...  ..  ..  ..  ...     ...  ...  ...   ...  ...  ...   
 475    Yuta Watanabe  BKN  14   5   9  255      22    4    6  66.7  ...    8   
 476     Zach Collins  SAS  14   5   9  265       9    4    6  66.7  ...    9   
 477      Zach LaVine  CHI  19   8  11  652     178   40   94  42.6  ...  128   
 478       Zeke Nnaji  DEN  16  10   6  166       3    0    1   0.0  ...    0   
 479  Zion Williamson  NOP  18  12   6  576     236   55  105  52.4  ...  161   
 
       PTS%  PASS  PASS%  

In [8]:
tables[1]
temp = tables[1]
temp.columns = temp.columns.droplevel() 
#temp = temp.drop(columns = ['Unnamed: 18_level_1','Unnamed: 19_level_1','Unnamed: 20_level_1', 'Unnamed: 21_level_1','Unnamed: 22_level_1'])
#temp
tables[1] = temp
tables[1]

,PLAYER,TEAM,AGE,GP,G,FREQ%,FGM,FGA,FG%,EFG%,...,2FG%,3FG FREQ%,3PM,3PA,3P%,Unnamed: 18_level_1,Unnamed: 19_level_1,Unnamed: 20_level_1,Unnamed: 21_level_1,Unnamed: 22_level_1
0,Royce O'Neale,BKN,29,25,25,57.1,57,125,45.6,66.8,...,57.1,53.9,53,118,44.9,NaN,NaN,NaN,NaN,NaN
1,Trae Young,ATL,24,21,21,24.7,51,113,45.1,59.3,...,65.5,18.4,32,84,38.1,NaN,NaN,NaN,NaN,NaN
2,Buddy Hield,IND,29,23,23,32.4,47,105,44.8,64.3,...,85.7,30.2,41,98,41.8,NaN,NaN,NaN,NaN,NaN
3,Nikola Vucevic,CHI,32,23,23,35.8,42,105,40.0,54.3,...,70.6,30.0,30,88,34.1,NaN,NaN,NaN,NaN,NaN
4,Tyrese Haliburton,IND,22,22,22,30.7,42,103,40.8,59.2,...,66.7,28.9,38,97,39.2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Devon Dotson,WAS,23,2,1,33.3,0,1,0.0,0.0,...,0.0,0.0,0,0,-,NaN,NaN,NaN,NaN,NaN
441,Marko Simonovic,CHI,23,1,1,100.0,0,1,0.0,0.0,...,-,100.0,0,1,0.0,NaN,NaN,NaN,NaN,NaN
442,Ryan Arcidiacono,NYK,28,6,1,33.3,0,1,0.0,0.0,...,-,33.3,0,1,0.0,NaN,NaN,NaN,NaN,NaN
443,Bogdan Bogdanovic,ATL,30,1,1,11.1,0,1,0.0,0.0,...,-,11.1,0,1,0.0,NaN,NaN,NaN,NaN,NaN


In [9]:
temp = temp.drop(columns = ['Unnamed: 18_level_1','Unnamed: 19_level_1','Unnamed: 20_level_1', 'Unnamed: 21_level_1','Unnamed: 22_level_1'])


In [10]:
tables[1] = temp
tables[1]

,PLAYER,TEAM,AGE,GP,G,FREQ%,FGM,FGA,FG%,EFG%,2FG FREQ%,2FGM,2FGA,2FG%,3FG FREQ%,3PM,3PA,3P%
0,Royce O'Neale,BKN,29,25,25,57.1,57,125,45.6,66.8,3.2,4,7,57.1,53.9,53,118,44.9
1,Trae Young,ATL,24,21,21,24.7,51,113,45.1,59.3,6.3,19,29,65.5,18.4,32,84,38.1
2,Buddy Hield,IND,29,23,23,32.4,47,105,44.8,64.3,2.2,6,7,85.7,30.2,41,98,41.8
3,Nikola Vucevic,CHI,32,23,23,35.8,42,105,40.0,54.3,5.8,12,17,70.6,30.0,30,88,34.1
4,Tyrese Haliburton,IND,22,22,22,30.7,42,103,40.8,59.2,1.8,4,6,66.7,28.9,38,97,39.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Devon Dotson,WAS,23,2,1,33.3,0,1,0.0,0.0,33.3,0,1,0.0,0.0,0,0,-
441,Marko Simonovic,CHI,23,1,1,100.0,0,1,0.0,0.0,0.0,0,0,-,100.0,0,1,0.0
442,Ryan Arcidiacono,NYK,28,6,1,33.3,0,1,0.0,0.0,0.0,0,0,-,33.3,0,1,0.0
443,Bogdan Bogdanovic,ATL,30,1,1,11.1,0,1,0.0,0.0,0.0,0,0,-,11.1,0,1,0.0


In [11]:
for i in range(len(name_list)):
    tables[i].to_csv('player_tracking/'+name_list[i]+'.csv',index = False)

In [12]:
tables[0].to_csv('player_tracking/pullup.csv',index = False)
tables[1].to_csv('player_tracking/cs.csv',index = False)